In [ ]:
import os
import glob
import dask
from dask.distributed import Client, progress, LocalCluster
from pyFIRS.wrappers import lastools
from pyFIRS.wrappers import fusion
from pyFIRS.utils import validation_summary, move_invalid_tiles, fname

### Setting up parallel computing using `dask.distributed`
`LAStools` offers native multi-core processing as an optional argument (`cores`) supplied to its command-line tools. `FUSION` command line tools do not. To enable parallel processing of `FUSION` commands, we'll use `dask.distributed` to schedule the processing of tiles in asynchronous parallel batches. This approach also offers us the ability to track progress using a progress bar.

You'll first need to launch a parallel computing cluster. 

In [ ]:
cluster=LocalCluster(scheduler_port=7001, diagnostics_port=7002)
c = Client(cluster)

At this point, you should also be able to view an interactive dashboard on port 7002. If you're executing this on a remote server, you'll need to set up port forward so you can view the dashboard on your local machine's browser. Once you've done that, or if you're processing on your own machine, you can view the dashboard at [http://localhost:7002/status](http://localhost:7002/status).

# Enough already, let's get to work with some lidar data
We'll define where we can find the binary executables for LAStools and FUSION command line tools.

In [ ]:
las = lastools.useLAStools('/storage/lidar/LAStools/bin')
fus = fusion.useFUSION('/storage/lidar/FUSION/')

In [ ]:
# data handling directories
WORKDIR = os.path.abspath('/storage/lidar/olc_metro_2014/')
SRC = os.path.join(WORKDIR, 'src')
src_tiles = glob.glob(os.path.join(SRC, '*.laz'))

# where we're going to put processed source tiles
RAW = os.path.join(WORKDIR, 'raw')

TARGET_EPSG = 6339 # utm 10N, NAD83_2011

print('Found {:,d} tiles in source directory:\n  {}'.format(len(src_tiles), SRC))

In [ ]:
# take a peak at info from a lidar source tile
info_proc = las.lasinfo(i=src_tiles[0],
                        echo=True)

In [ ]:
num_cores = len(c.ncores()) # identify how many workers we have

# push our working directories and wrapper classes to the workers on the cluster as well
c.scatter([WORKDIR, SRC, RAW, las, fus, TARGET_EPSG, num_cores], broadcast=True);

## Get the raw data into our working directory
First, move the tiles over to our working directory.

When we define functions using the `dask.delayed` decorator, the function will have 'lazy' instead of 'eager' execution. We can map the function to a list of inputs and it will not execute for any of them until we ask for results to be computed. When we use the `compute()` method for the client managing the scheduler that sends jobs to the workers, it then starts running the jobs.

In [ ]:
@dask.delayed
def import_tile(tile_id): # the function we'll map to a list of inputs
    INFILE = os.path.join(SRC, tile_id + '.laz')
    OUTFILE = os.path.join(RAW, tile_id + '.laz')
    
    if os.path.exists(OUTFILE):
        pass
    else:
        proc_import =  las.las2las(i=INFILE,
                                   drop_withheld=True, # drop any points flagged as withheld by vendor
                                   drop_class=(7,18), # drop any points classified as noise by vendor
#                                    epsg=src_epsg, # specify the source lidar projection in case it isn't automatically recognized
                                   nad83_2011=True, # original data in nad83_2011 datum
                                   target_epsg=TARGET_EPSG, # reproject to utm zone 10 N,
                                   dont_remove_empty_files=True,
                                   odir=RAW,
                                   olaz=True) # compress .laz file output
    return tile_id

Next, validate that the data match LAS specifications and have not been corrupted.

In [ ]:
@dask.delayed
def validate(tile_id):
    INFILE = os.path.join(RAW, tile_id + '.laz')
    OUTFILE = os.path.join(RAW, tile_id + '.xml')
    
    if os.path.exists(OUTFILE):
        pass
    else:
        proc_validate = las.lasvalidate(i=INFILE,
                                        o=OUTFILE)
    return tile_id

Next, create spatial indexes for the input files to allow fast spatial queries (which are used, for example, when retiling and adding buffers).

In [ ]:
@dask.delayed
def make_index(tile_id): # the function we'll map to a list of inputs
    INFILE = os.path.join(RAW, tile_id + '.laz')
    OUTFILE = os.path.join(RAW, tile_id + '.lax')
    
    if os.path.exists(OUTFILE): 
        pass
    else:
        proc_index = las.lasindex(i=INFILE)
    
    return tile_id

## Hand-build the computational graph
Define the recipe for computations.

In [ ]:
# tile_ids = [fname(tile) for tile in src_tiles]
tile_ids = [fname(tile) for tile in extra_orig]

get_data = {} # a dictionary that will be used to define dask's computational graph
for tile in tile_ids:
    get_data['import-{}'.format(tile)]=(import_tile, tile)
    get_data['validate-{}'.format(tile)]=(validate, 'import-{}'.format(tile))
    get_data['index-{}'.format(tile)]=(make_index, 'validate-{}'.format(tile))
    
# this empty function will be added to recipe for computations
# it will be defined to depend upon all previous steps being completed
@dask.delayed
def done_importing(*args, **kwargs):
    return

get_data['done_importing']=(done_importing, ['index-{}'.format(tile) for tile in tile_ids])

In [ ]:
get_data_graph = c.get(get_data, 'done_importing') # builds the computational graph

In [ ]:
get_data_results = c.compute(get_data_graph) # starts executing it

In [ ]:
progress(get_data_results) # progress bars

In [ ]:
# c.cancel(get_data_results)

In [ ]:
validation_summary(xml_dir=RAW, verbose=True)

In [ ]:
# move_invalid_tiles(xml_dir=RAW, dest_dir=os.path.join(RAW, 'invalid'))

In [ ]:
# c.close()
# cluster.close()